# Assignment 3 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Personal Details:

In [ ]:
# Details Student 1:
Ofir Salomon
304845688
ofir_sela@hotmail.com
# Details Student 2:
Itzik Ben Harush
312429525
itzikbena19@gmail.com


## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [1]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [2]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [ ]:
# word net installation:

# unmark if you want to use and need to install
# !pip install wn
# !python -m wn download omw-he:1.4

In [3]:
# word net import:

# unmark if you want to use:
import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [ ]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
# !pip install hebrew_tokenizer

In [4]:
# Hebrew tokenizer import:

# unmark if you want to use:
import hebrew_tokenizer as ht

C:\Users\User


### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [5]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [6]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [7]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [8]:
def hebrew_tokenizer(text):
    hebrew_text = ' '.join(re.findall(r'[\u0590-\u05FF]+', text))
    tokens = hebrew_text.split()
    return tokens

gender_normlize = {'m': 1, 'f': 0}

df_train['gender'] = df_train['gender'].map(gender_normlize)
X_train = df_train['story'] 
y_train = df_train['gender']

In [9]:
normalizer = preprocessing.Normalizer(norm='l2')

vec_list = [
    ("CountVectorizer", CountVectorizer(tokenizer=hebrew_tokenizer, max_df=0.75, ngram_range=(1, 2))),
    ("TfidfVectorizer", TfidfVectorizer(tokenizer=hebrew_tokenizer, max_df=0.75, ngram_range=(1, 2)))
]

In [10]:
model_list = [
    ('Decision Tree', DecisionTreeClassifier(max_depth=10, min_samples_split=4)),
]

best_f1_score = 0
best_vectorizer = None

for vec_name, vectorizer in vec_list:
    total_f1_score = 0
    num_models = 0
    
    for model_name, model in model_list:
        pipeline = Pipeline([
            ('vec', vectorizer),
            ('clf', model)
        ])
        
        avg_f1_score = np.mean(cross_val_score(pipeline, X_train, y_train, cv=5, scoring='f1_macro'))
        total_f1_score += avg_f1_score
        num_models += 1

        print(f"{vec_name} with {model_name} - Average F1 score: {avg_f1_score}")

    avg_f1_across_models = total_f1_score / num_models
    
    if avg_f1_across_models > best_f1_score:
        best_f1_score = avg_f1_across_models
        best_vectorizer = vectorizer

print(f"Best Vectorizer: {best_vectorizer}, Average F1 score across models: {best_f1_score}")

CountVectorizer with Decision Tree - Average F1 score: 0.564649829925908
TfidfVectorizer with Decision Tree - Average F1 score: 0.567522005976377
Best Vectorizer: TfidfVectorizer(max_df=0.75, ngram_range=(1, 2),
                tokenizer=<function hebrew_tokenizer at 0x0000020494B46C00>), Average F1 score across models: 0.567522005976377


In [11]:
model_list = [

    ('DecisionTree', Pipeline([
        ('tfidf', TfidfVectorizer(analyzer=hebrew_tokenizer)),
        ('normalizer', normalizer),
        ('clf', DecisionTreeClassifier())
    ]), {
        'tfidf__max_df': [0.5, 0.75, 1.0],
        'clf__criterion': ['gini', 'entropy'],
        'clf__max_depth': [None, 10, 20],
        'clf__min_samples_split': [2, 5, 10],
        'clf__min_samples_leaf': [1, 2, 4]
    }),
    
    ('SGDClassifier', Pipeline([
        ('tfidf', TfidfVectorizer(analyzer=hebrew_tokenizer)),
        ('normalize', preprocessing.Normalizer()),
        ('clf', SGDClassifier(max_iter=1000, random_state=42))
    ]), {
        'tfidf__max_df': [0.5, 0.75, 1.0],
        'clf__alpha': [0.0001, 0.001, 0.01, 0.1],
        'clf__penalty': ['l1', 'l2', 'elasticnet'],
        'clf__max_iter': [500, 1000],
        'clf__learning_rate': ['optimal', 'adaptive'],
        'clf__eta0': [0.01, 0.1, 0.5],
        'clf__n_iter_no_change': [5, 10]
    }),
    
    ('Naive Bayes', Pipeline([
        ('tfidf', TfidfVectorizer(analyzer=hebrew_tokenizer)),
        ('normalize', preprocessing.Normalizer(norm='l2')),
        ('clf', MultinomialNB())
    ]), {
        'tfidf__max_df': [0.5, 0.75, 1.0],
        'clf__alpha': [0.0001, 0.001, 0.01, 0.1],
        'clf__fit_prior': [True, False]
    }),

    ('Linear SVC', Pipeline([
        ('tfidf', TfidfVectorizer(analyzer=hebrew_tokenizer)),
        ('normalize', preprocessing.Normalizer(norm='l2')),
        ('classifier', LinearSVC())
    ]), {
        'tfidf__max_df': [0.5, 0.75, 1.0],
        'classifier__C': [0.1, 1, 10],
        'classifier__penalty': ['l1', 'l2'],
        'classifier__loss': ['hinge', 'squared_hinge'],
        'classifier__tol': [1e-4, 1e-3, 1e-2],
        'classifier__max_iter': [1000, 2000, 3000]
    }),

]


In [12]:
best_model = None
best_f1_score = 0

for model_name, pipeline, param_grid in model_list:
    print(f"Evaluating the {model_name}")

    grid_search = GridSearchCV(pipeline, param_grid, cv=10, scoring='f1_macro', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    print(f"Best hyperparameters for {model_name}: {grid_search.best_params_}")
    print(f"Cross-validation F1 score for {model_name}: {grid_search.best_score_}")

    y_pred = grid_search.predict(X_train)
    cm = confusion_matrix(y_train, y_pred)
    print(f"Confusion Matrix for {model_name}:")
    print(cm)

    print(f"Vectorizer used: {pipeline.named_steps['tfidf'] if 'tfidf' in pipeline.named_steps else pipeline.named_steps['countvectorizer']}")
    
    print("------------------------")

    if grid_search.best_score_ > best_f1_score:
        best_f1_score = grid_search.best_score_
        best_model = grid_search.best_estimator_
        best_vectorizer = pipeline.named_steps['tfidf'] if 'tfidf' in pipeline.named_steps else pipeline.named_steps['countvectorizer']

print(f"Best model: {best_model.named_steps['clf']}")
print(f"Best F1 score: {best_f1_score}")
print(f"Best Vectorizer: {best_vectorizer}")

Evaluating the DecisionTree


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(analyzer=<function hebrew_tokenizer at 0x0000020494B46C00>)),
                                       ('normalizer', Normalizer()),
                                       ('clf', DecisionTreeClassifier())]),
             n_jobs=-1,
             param_grid={'clf__criterion': ['gini', 'entropy'],
                         'clf__max_depth': [None, 10, 20],
                         'clf__min_samples_leaf': [1, 2, 4],
                         'clf__min_samples_split': [2, 5, 10],
                         'tfidf__max_df': [0.5, 0.75, 1.0]},
             scoring='f1_macro')

Best hyperparameters for DecisionTree: {'clf__criterion': 'entropy', 'clf__max_depth': 20, 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 10, 'tfidf__max_df': 0.75}
Cross-validation F1 score for DecisionTree: 0.6239906243384319
Confusion Matrix for DecisionTree:
[[150  28]
 [  8 567]]
Vectorizer used: TfidfVectorizer(analyzer=<function hebrew_tokenizer at 0x0000020494B46C00>)
------------------------
Evaluating the SGDClassifier


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(analyzer=<function hebrew_tokenizer at 0x0000020494B46C00>)),
                                       ('normalize', Normalizer()),
                                       ('clf',
                                        SGDClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid={'clf__alpha': [0.0001, 0.001, 0.01, 0.1],
                         'clf__eta0': [0.01, 0.1, 0.5],
                         'clf__learning_rate': ['optimal', 'adaptive'],
                         'clf__max_iter': [500, 1000],
                         'clf__n_iter_no_change': [5, 10],
                         'clf__penalty': ['l1', 'l2', 'elasticnet'],
                         'tfidf__max_df': [0.5, 0.75, 1.0]},
             scoring='f1_macro')

Best hyperparameters for SGDClassifier: {'clf__alpha': 0.0001, 'clf__eta0': 0.01, 'clf__learning_rate': 'optimal', 'clf__max_iter': 500, 'clf__n_iter_no_change': 10, 'clf__penalty': 'l1', 'tfidf__max_df': 0.75}
Cross-validation F1 score for SGDClassifier: 0.7093719026285026
Confusion Matrix for SGDClassifier:
[[177   1]
 [  0 575]]
Vectorizer used: TfidfVectorizer(analyzer=<function hebrew_tokenizer at 0x0000020494B46C00>)
------------------------
Evaluating the Naive Bayes


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(analyzer=<function hebrew_tokenizer at 0x0000020494B46C00>)),
                                       ('normalize', Normalizer()),
                                       ('clf', MultinomialNB())]),
             n_jobs=-1,
             param_grid={'clf__alpha': [0.0001, 0.001, 0.01, 0.1],
                         'clf__fit_prior': [True, False],
                         'tfidf__max_df': [0.5, 0.75, 1.0]},
             scoring='f1_macro')

Best hyperparameters for Naive Bayes: {'clf__alpha': 0.01, 'clf__fit_prior': False, 'tfidf__max_df': 0.5}
Cross-validation F1 score for Naive Bayes: 0.5914536652376198
Confusion Matrix for Naive Bayes:
[[178   0]
 [  2 573]]
Vectorizer used: TfidfVectorizer(analyzer=<function hebrew_tokenizer at 0x0000020494B46C00>)
------------------------
Evaluating the Linear SVC


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(analyzer=<function hebrew_tokenizer at 0x0000020494B46C00>)),
                                       ('normalize', Normalizer()),
                                       ('classifier', LinearSVC())]),
             n_jobs=-1,
             param_grid={'classifier__C': [0.1, 1, 10],
                         'classifier__loss': ['hinge', 'squared_hinge'],
                         'classifier__max_iter': [1000, 2000, 3000],
                         'classifier__penalty': ['l1', 'l2'],
                         'classifier__tol': [0.0001, 0.001, 0.01],
                         'tfidf__max_df': [0.5, 0.75, 1.0]},
             scoring='f1_macro')

Best hyperparameters for Linear SVC: {'classifier__C': 10, 'classifier__loss': 'squared_hinge', 'classifier__max_iter': 1000, 'classifier__penalty': 'l2', 'classifier__tol': 0.0001, 'tfidf__max_df': 0.75}
Cross-validation F1 score for Linear SVC: 0.6402735092739962
Confusion Matrix for Linear SVC:
[[177   1]
 [  0 575]]
Vectorizer used: TfidfVectorizer(analyzer=<function hebrew_tokenizer at 0x0000020494B46C00>)
------------------------
Best model: SGDClassifier(eta0=0.01, max_iter=500, n_iter_no_change=10, penalty='l1',
              random_state=42)
Best F1 score: 0.7093719026285026
Best Vectorizer: TfidfVectorizer(analyzer=<function hebrew_tokenizer at 0x0000020494B46C00>)


In [14]:
best_model.fit(X_train, y_train)

f1_scores = cross_val_score(best_model, X_train, y_train, cv=10, scoring='f1_macro')

average_f1_score = np.mean(f1_scores)

print("Average F1 score:", average_f1_score)

predicted_labels = best_model.predict(df_test["story"])

predicted_gender = np.where(predicted_labels == 1, 'm', 'f')

df_predicted = pd.DataFrame({'test_example_id': df_test['test_example_id'], 'predicted_gender': predicted_gender})

print(df_predicted)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(analyzer=<function hebrew_tokenizer at 0x0000020494B46C00>,
                                 max_df=0.75)),
                ('normalize', Normalizer()),
                ('clf',
                 SGDClassifier(eta0=0.01, max_iter=500, n_iter_no_change=10,
                               penalty='l1', random_state=42))])

Average F1 score: 0.7093719026285026
     test_example_id predicted_gender
0                  0                m
1                  1                m
2                  2                m
3                  3                m
4                  4                f
..               ...              ...
318              318                m
319              319                m
320              320                m
321              321                m
322              322                m

[323 rows x 2 columns]


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [15]:
df_predicted.to_csv('classification_results.csv',index=False)